In [1]:
# INSTALLING PACKAGES
!pip install -U torch torchvision torchaudio
!pip install -U ultralytics


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [1]:
# CLONING YOLOv5 REPOSITORY
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
!pip install -r requirements.txt

fatal: destination path 'yolov5' already exists and is not an empty directory.
/content/yolov5


In [2]:
# DOWNLOADING FRUIT DATSET FROM KAGGLE
!pip install opendatasets
import opendatasets as od

In [3]:
od.download("https://www.kaggle.com/moltean/fruits")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: "muneeba44"
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/moltean/fruits


100%|██████████| 1.28G/1.28G [00:09<00:00, 150MB/s]


In [4]:
import os
import shutil

def fruits_data():
    dataset_path = '/content/yolov5/fruits/fruits-360_dataset/fruits-360/Training'
    output_path = '/content/yolov5/fruits_dataset'

    if not os.path.exists(output_path):
        os.makedirs(output_path)
        os.makedirs(os.path.join(output_path, 'images'))
        os.makedirs(os.path.join(output_path, 'labels'))

    class_names = os.listdir(dataset_path)

    with open(os.path.join(output_path, 'classes.txt'), 'w') as f:
        for class_name in class_names:
            f.write(f"{class_name}\n")

    for class_id, class_name in enumerate(class_names):
        class_path = os.path.join(dataset_path, class_name)
        image_files = os.listdir(class_path)

        for image_file in image_files:
            image_path = os.path.join(class_path, image_file)
            label_path = os.path.join(output_path, 'labels', image_file.replace('.jpg', '.txt'))

            # Move image to images folder
            shutil.copy(image_path, os.path.join(output_path, 'images', image_file))

            # Create label file
            with open(label_path, 'w') as lf:
                # Assuming entire image is the crop, this is a simplification
                lf.write(f"{class_id} 0.5 0.5 1.0 1.0\n")

    return output_path, len(class_names), class_names

In [5]:
dataset_path, num_classes, class_names = fruits_data()

In [6]:
# CREATING A DATA CONFIGURATION FILE FOR YOLOv5
data_yaml = f"""
train: {dataset_path}/images
val: {dataset_path}/images
nc: {num_classes}
names: {class_names}
"""

with open('fruits_data.yaml', 'w') as f:
    f.write(data_yaml)

# TRAINING YOLOV5 ON FRUITS DATASET
!python train.py --img 640 --batch 16 --epochs 5 --data fruits_data.yaml --weights yolov5s.pt


2024-05-26 05:26:22.427995: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-26 05:26:22.428047: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-26 05:26:22.585823: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5s.pt, cfg=, data=fruits_data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=5, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False,